# 5.5 读写文件
+ 有时我们希望保存训练的模型， 以备将来在各种环境中使用（比如在部署中进行预测）
+ 当运行一个耗时较长的训练过程时， 最佳的做法是定期保存中间结果， 以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。

总结:
+ 1. 可以通过`load()`和`save()`来进行单个张量的保存
+ 2. 可以通过`save(net.state_dict(),'filename') ` `load_state_dict('filename')`进行整个模型参数的保存与读取

## 5.5.1 加载和保存张量
单个张量直接调用load和save函数分别读写，两个函数都需要我们提供一个名称？

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
# x作为输出保存到x-file文件当中
torch.save(x, 'x-file')

In [2]:
# 存储在文件中的数据读回内存
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

存储一个张量列表 然后读回内存

In [3]:
y = torch.arange(4)
torch.save([x,y],'./data/x-files')

In [5]:
# 读取张量列表
x2,y2 = torch.load('./data/x-files')
print(x2)
print(y2)


tensor([0, 1, 2, 3])
tensor([0, 1, 2, 3])


写入或读取字符串映射到张量的字典

In [6]:
mydict = {'x':x,'y':y}
torch.save(mydict,'./data/mydict')
mydict2 = torch.load('./data/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0, 1, 2, 3])}

## 5.5.2 加载和保存模型的参数
深度学习框架提供了内置函数来保存和加载整个网络，但是保存仅仅保存参数而不是整个模型

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [8]:
# 模型的参数存储在一个叫做“mlp.params”的文件中
torch.save(net.state_dict(),'./data/mpl.params')

In [9]:
clone = MLP()
# 从文件当中加载参数
clone.load_state_dict(torch.load('./data/mpl.params'))
# 设置模型为评估模式 关闭一些计算 更快速更稳定
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [10]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])